<a href="https://colab.research.google.com/github/Vicente-Figueroa/IBM-MachineLearning/blob/main/Metallica_lstm_songs_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metallica song's name genetaror
### Objective:
The purpose of this work is to create a LSTM neural net to preview a workload of data scientist

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import os

In [ ]:
step_length = 1 # The step length we take to get our samples from our corpus
epochs = 50 # Number of times we train on our full data  50
batch_size = 32 # Data samples in each training step
latent_dim = 64 # Size of our LSTM
dropout_rate = 0.2 # Regularization with dropout
model_path = os.path.realpath('./metallica_gen_model.h5') # Location for the model
load_model = False # Enable loading model from disk
store_model = True # Store model to disk after training
verbosity = 1 # Print result for each epoch
gen_amount = 10 # How many

In [ ]:
df= pd.read_csv('metallica.csv', header=None, sep='\t')

input_names = df[0]

In [ ]:
# Make it all to a long string
concat_names = '\n'.join(input_names).lower()

# Find all unique characters by using set()
chars = sorted(list(set(concat_names)))
num_chars = len(chars)

# Build translation dictionaries, 'a' -> 0, 0 -> 'a'
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))


# Use longest name length as our sequence window
max_sequence_length = max([len(name) for name in input_names])

print('Total chars: {}'.format(num_chars))
print('Corpus length:', len(concat_names))
print('Number of names: ', len(input_names))
print('Longest name: ', max_sequence_length)

Total chars: 43
Corpus length: 3236
Number of names:  206
Longest name:  35


In [ ]:
sequences = []
next_chars = []

# Loop over our data and extract pairs of sequances and next chars
for i in range(0, len(concat_names) - max_sequence_length, step_length):
    sequences.append(concat_names[i: i + max_sequence_length])
    next_chars.append(concat_names[i + max_sequence_length])

num_sequences = len(sequences)

print('Number of sequences:', num_sequences)
print('First 10 sequences and next chars:')
for i in range(10):
    print('X=[{}] y=[{}]'.replace('\n', ' ').format(sequences[i], next_chars[i]).replace('\n', ' '))

Number of sequences: 3201
First 10 sequences and next chars:
X=[2 x 4 53rd & 3rd ain't my bitch all] y=[ ]
X=[ x 4 53rd & 3rd ain't my bitch all ] y=[d]
X=[x 4 53rd & 3rd ain't my bitch all d] y=[a]
X=[ 4 53rd & 3rd ain't my bitch all da] y=[y]
X=[4 53rd & 3rd ain't my bitch all day] y=[ ]
X=[ 53rd & 3rd ain't my bitch all day ] y=[a]
X=[53rd & 3rd ain't my bitch all day a] y=[n]
X=[3rd & 3rd ain't my bitch all day an] y=[d]
X=[rd & 3rd ain't my bitch all day and] y=[ ]
X=[d & 3rd ain't my bitch all day and ] y=[a]


In [ ]:
X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=np.bool)
Y = np.zeros((num_sequences, num_chars), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for j, char in enumerate(sequence):
        X[i, j, char2idx[char]] = 1
        Y[i, char2idx[next_chars[i]]] = 1

print('X shape: {}'.format(X.shape))
print('Y shape: {}'.format(Y.shape))

X shape: (3201, 35, 43)
Y shape: (3201, 43)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
# making the NN
model = Sequential()

# Add the LSTM layer 
model.add(LSTM(latent_dim,
               input_shape=(max_sequence_length, num_chars),
               recurrent_dropout=dropout_rate))

# Output layer dense and softmax activation
model.add(Dense(units=num_chars, activation='softmax'))

# Optmizer in RMS
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                27648     
                                                                 
 dense (Dense)               (None, 43)                2795      
                                                                 
Total params: 30,443
Trainable params: 30,443
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
if load_model:
    model.load_weights(model_path)
else:
    start = time.time()
    print('Start training for {} epochs'.format(epochs))
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbosity)
    end = time.time()
    print('Finished training - time elapsed:', (end - start)/60, 'min')
if store_model:
    print('Storing model at:', model_path)
    model.save(model_path)

Start training for 50 epochs
Epoch 1/50
101/101 [==============================] - 12s 81ms/step - loss: 2.9713
Epoch 2/50
101/101 [==============================] - 7s 69ms/step - loss: 2.5909
Epoch 3/50
101/101 [==============================] - 4s 38ms/step - loss: 2.4254
Epoch 4/50
101/101 [==============================] - 4s 38ms/step - loss: 2.3238
Epoch 5/50
101/101 [==============================] - 4s 38ms/step - loss: 2.2185
Epoch 6/50
101/101 [==============================] - 4s 38ms/step - loss: 2.1391
Epoch 7/50
101/101 [==============================] - 4s 38ms/step - loss: 2.0415
Epoch 8/50
101/101 [==============================] - 4s 38ms/step - loss: 1.9677
Epoch 9/50
101/101 [==============================] - 4s 38ms/step - loss: 1.8974
Epoch 10/50
101/101 [==============================] - 4s 38ms/step - loss: 1.8187
Epoch 11/50
101/101 [==============================] - 4s 37ms/step - loss: 1.7819
Epoch 12/50
101/101 [==============================] - 4s 38ms/ste

In [ ]:
# Start sequence generation from end of the input sequence
sequence = concat_names[-(max_sequence_length - 1):] + '\n'

new_names = []
print('{} new names are being generated'.format(gen_amount))

while len(new_names) < gen_amount:
    # Vectorize sequence for prediction
    x = np.zeros((1, max_sequence_length, num_chars))
    for i, char in enumerate(sequence):
        x[0, i, char2idx[char]] = 1

    # Sample next char from predicted probabilities
    probs = model.predict(x, verbose=0)[0]
    probs /= probs.sum()
    next_idx = np.random.choice(len(probs), p=probs)
    next_char = idx2char[next_idx]
    sequence = sequence[1:] + next_char

    # New line means we have a new name
    if next_char == '\n':
        gen_name = [name for name in sequence.split('\n')][1]
        
        # Never start name with two identical chars, could probably also
        if len(gen_name) > 2 and gen_name[0] == gen_name[1]:
            gen_name = gen_name[1:]
        
        # Discard all names that are too short
        if len(gen_name) > 2:
            # Only allow new and unique names
            if gen_name not in np.concatenate((input_names, new_names), axis=0):
                new_names.append(gen_name.capitalize())
        
        if 0 == (len(new_names) % (gen_amount/ 10)):
            print('Generated {}'.format(len(new_names)))

10 new names are being generated
Generated 1
Generated 2
Generated 3
Generated 4
Generated 5
Generated 6
Generated 7
Generated 8
Generated 9
Generated 10


In [ ]:
print_first_n = min(10, gen_amount)
print('First {} generated names:'.format(print_first_n))
for name in new_names[:print_first_n]:
    print(name)

First 10 generated names:
Would?
Diance
Diem i see
Moners in the bone
My vmanos
My vmanos
Homors of the nigst
Whings jast cait
Whings jast cait
Wasting case
